<a href="https://colab.research.google.com/github/shazzad-hasan/practice-deep-learning-with-pytorch/blob/main/image_classification/cat_vs_dog.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this notebook, we will train a ConvNet to classify whether images contain either a dog or a cat using [Dogs vs. Cats](https://www.kaggle.com/c/dogs-vs-cats/overview) dataset. The dataset contains 25,000 images of dogs and cats (12,500 from each class).

The process will be broken down into the following steps:

    1. Load and visualize the dataset
    2. Define a pre-trained model
        1. Load in a pre-trained model
        2. Freeze all the parameters, so that the network acts as a fixed feature extractor
        3. Remove the last layer
        4. Replace the last layer with a linear classifier
    3. Define a loss function and optimizer
    4. Train the model on the training dataset
    5. Evaluate the performance of the trained model on the test dataset



In [ ]:
# upload kaggle API key from your local machine
from google.colab import files
files.upload()

In [ ]:
# make a kaggle dir, copy the API key to it
# and make sure the file in only readable by yourself (chmod 600)
!mkdir ~/.kaggle 
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
# use API command to download the dataset
!kaggle competitions download -c dogs-vs-cats

In [ ]:
# uncompress the dataset
!unzip -qq dogs-vs-cats
!unzip -qq train.zip
!unzip -qq test1.zip

In [ ]:
# import required libraries
import torch
import torchvision

import os
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# check if cuda is available
train_on_gpu = torch.cuda.is_available()

if train_on_gpu:
  print("CUDA is available!")
else:
  print("CUDA is not available!")

device = torch.device("cuda") if train_on_gpu else torch.device("cpu")

In [ ]:
print(len(os.listdir("/content/train")))
print(len(os.listdir("/content/test1")))

In [ ]:
# import required libraries
import os, shutil, pathlib

# path to dir where original dataset was uncompressed
original_train_dir = pathlib.Path("train")
original_test_dir = pathlib.Path("test1")

# dir of the smaller dataset
new_base_dir = pathlib.Path("cats_vs_dogs")

def make_train_valid_subset(subset_name, original_dir, new_base_dir, start_index, end_index):
  for category in ("cat", "dog"):
    dir = new_base_dir / subset_name / category
    os.makedirs(dir)
    fnames = [f"{category}.{i}.jpg" for i in range(start_index, end_index)]
    for fname in fnames:
      shutil.copyfile(src = original_dir / fname, dst = dir / fname)

def make_test_subset(subset_name, original_dir, new_base_dir, start_index, end_index):
  for category in ("cat", "dog"):
    dir = new_base_dir / subset_name / category
    os.makedirs(dir)
    fnames = [f"{i}.jpg" for i in range(start_index, end_index)]
    for fname in fnames:
      shutil.copyfile(src = original_dir / fname, dst = dir / fname)

# make 3 subsets: trai, validation, test
make_train_valid_subset("train", original_train_dir, new_base_dir, start_index=0, end_index=2000)
make_train_valid_subset("valid", original_train_dir, new_base_dir, start_index=2000, end_index=3000)
make_test_subset("test", original_test_dir, new_base_dir, start_index=1, end_index=1000)

In [ ]:
def make_test_subset(subset_name, original_dir, new_base_dir, start_index, end_index):
  for category in ("cat", "dog"):
    dir = new_base_dir / subset_name / category
    os.makedirs(dir)
    fnames = [f"{i}.jpg" for i in range(start_index, end_index)]
    for fname in fnames:
      shutil.copyfile(src = original_dir / fname, dst = dir / fname)

make_test_subset("test", original_test_dir, new_base_dir, start_index=1, end_index=1000)

In [ ]:
train_dir = os.path.join(new_base_dir, "train")
valid_dir = os.path.join(new_base_dir, "valid")
test_dir = os.path.join(new_base_dir, "test")

In [ ]:
from torchvision import datasets
import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler

# load and transform data using ImageFolder
data_transform = transforms.Compose([
        transforms.Resize((244, 244)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])
])

train_data = datasets.ImageFolder(train_dir, transform=data_transform)
valid_data = datasets.ImageFolder(valid_dir, transform=data_transform)
test_data = datasets.ImageFolder(test_dir, transform=data_transform)

print("Number of training images: ", len(train_data))
print("Number of validation images: ", len(valid_data))
print("Number of test images: ", len(test_data))